# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcd106d6bb0>
├── 'a' --> tensor([[-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388]])
└── 'x' --> <FastTreeValue 0x7fcd10737d30>
    └── 'c' --> tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                        [ 0.1161, -1.4425, -0.0924,  0.6024],
                        [ 1.5269,  0.2550, -0.0057, -0.4908]])

In [4]:
t.a

tensor([[-1.0418,  0.8406,  0.3517],
        [-0.2532, -0.7897,  0.6388]])

In [5]:
%timeit t.a

62.3 ns ± 0.0749 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcd106d6bb0>
├── 'a' --> tensor([[-0.4408, -0.4938, -1.1471],
│                   [-0.7973,  0.0683,  0.0581]])
└── 'x' --> <FastTreeValue 0x7fcd10737d30>
    └── 'c' --> tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                        [ 0.1161, -1.4425, -0.0924,  0.6024],
                        [ 1.5269,  0.2550, -0.0057, -0.4908]])

In [7]:
%timeit t.a = new_value

69.6 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0418,  0.8406,  0.3517],
               [-0.2532, -0.7897,  0.6388]]),
    x: Batch(
           c: tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908]]),
       ),
)

In [10]:
b.a

tensor([[-1.0418,  0.8406,  0.3517],
        [-0.2532, -0.7897,  0.6388]])

In [11]:
%timeit b.a

66.4 ns ± 2.72 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0896, -0.2012,  0.4136],
               [-0.0924,  0.1804, -0.1268]]),
    x: Batch(
           c: tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908]]),
       ),
)

In [13]:
%timeit b.a = new_value

460 ns ± 27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

809 ns ± 0.172 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 37.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 6.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 457 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcc6cfe69d0>
├── 'a' --> tensor([[[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]],
│           
│                   [[-1.0418,  0.8406,  0.3517],
│                    [-0.2532, -0.7897,  0.6388]]])
└── 'x' --> <FastTreeValue 0x7fcc5b98c8b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

28.6 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcd106d2370>
├── 'a' --> tensor([[-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388],
│                   [-1.0418,  0.8406,  0.3517],
│                   [-0.2532, -0.7897,  0.6388]])
└── 'x' --> <FastTreeValue 0x7fcc5ba054f0>
    └── 'c' --> tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                        [ 0.1161, -1.4425, -0.0924,  0.6024],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 73.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1307, -0.3262,  0.1260,  0.6743],
                       [ 0.1161, -1.4425, -0.0924,  0.6024],
                       [ 1.5269,  0.2550, -0.0057, -0.4908]],
              
                      [[-0.1307, -0.3262,  0.1260,  0.6743],
                       [ 0.1161, -1.4425, -0.0924,  0.6024],
                       [ 1.5269,  0.2550, -0.0057, -0.4908]],
              
                      [[-0.1307, -0.3262,  0.1260,  0.6743],
                       [ 0.1161, -1.4425, -0.0924,  0.6024],
                       [ 1.5269,  0.2550, -0.0057, -0.4908]],
              
                      [[-0.1307, -0.3262,  0.1260,  0.6743],
                       [ 0.1161, -1.4425, -0.0924,  0.6024],
                       [ 1.5269,  0.2550, -0.0057, -0.4908]],
              
                      [[-0.1307, -0.3262,  0.1260,  0.6743],
                       [ 0.1161, -1.4425, -0.0924,  0.6024],
                       [ 1.5269,  0.2550, -0.0057, -0.4908]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908],
                      [-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908],
                      [-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908],
                      [-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908],
                      [-0.1307, -0.3262,  0.1260,  0.6743],
                      [ 0.1161, -1.4425, -0.0924,  0.6024],
                      [ 1.5269,  0.2550, -0.0057, -0.4908],
                      [-0.1307, -0.3262,  0.1260,  0.6743],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

292 µs ± 2.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
